In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.stats import pearsonr 
import tensorflow as tf

In [ ]:
def getCorr(x,y):
    corr, _ = pearsonr(x, y) 
    return corr

In [ ]:
def getSlope(df):
    return abs(df['slope'])

In [ ]:
dftrain = pd.read_csv('../input/dont-overfit-ii/train.csv') # train
dftest = pd.read_csv('../input/dont-overfit-ii/test.csv') # test

In [ ]:
dftrain

In [ ]:
plt.scatter(dftrain['299'],dftrain['1'])
plt.title('My PCA graph')
plt.xlabel('0 -{0}%'.format(dftrain['299']))
plt.ylabel('target -{0}%'.format(dftrain['1']))

In [ ]:
y=sns.regplot(x='1',y='target',data=dftrain)

In [ ]:
detailed_features=[]
for feature in range(300):
    y=sns.regplot(x=str(feature),y='target',data=dftrain)
    detailed_features.append({'feature':feature,'slope':getCorr(dftrain['target'],dftrain[str(feature)])})

In [ ]:
detailed_features.sort(key=getSlope,reverse=True)

In [ ]:
features_to_save=[]
for iteration in range(6):
    features_to_save.append(detailed_features[iteration]['feature'])

In [ ]:
def getDfProcessed(df_toProcess,features_to_save):
    df_processed = pd.DataFrame() 
    for iteration in features_to_save:
        feature = df_toProcess[str(iteration)].values
        df_processed[str(iteration)] = feature 
    return df_processed

In [ ]:
dftrain_processed = getDfProcessed(dftrain,features_to_save)

In [ ]:
dftest_processed = getDfProcessed(dftest,features_to_save)

In [ ]:
y_train = dftrain.pop('target')

In [ ]:

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(7, activation=tf.nn.relu),  
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])


In [ ]:
model.fit(dftrain_processed, y_train, epochs=10)

In [ ]:
predictions_list = list(model.predict(dftest_processed))

In [ ]:
def getTarget(df):
    return df[1]

In [ ]:
def getPredictions(prediction_list):
    prediction_list_processced = []
    for iteration in prediction_list :
        prediction_list_processced.append(int(round(iteration[1])))
    return prediction_list_processced

In [ ]:
dfsubmission = pd.DataFrame() 
dfsubmission['id'] = dftest['id'] 
dfsubmission['target'] = getPredictions(predictions_list)

In [ ]:
dfsubmission

In [ ]:
dfsubmission.to_csv('submission.csv', index=False)  
